# Introduction

 In this assignment, we will delve into dimensionality reduction and unsupervised learning
 tasks. Firstly, we should preprocess the provided dataset to prepare it for analysis. Next, we
 will apply dimensionality reduction techniques to simplify the dataset's complexity. Then, we
 will use unsupervised learning algorithms to tackle the task. Finally, we evaluate and analyze
 the results for comparison.

## Dataset

 In 2014, some researchers published an article called "Impact of c1HbA Measurement on
 Hospital Readmission Rates: Analysis of 70,000 Clinical Database Patient Records." They
 gathered data on diabetic patients from many hospitals and clinics in America. Some of this
 data, about 200,000 items with 50 features, has been shared with the public in a way that
 keeps people's identities private

# Implementation 

## Import Libraries

In [161]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import re


## 1.Preprocessing

Data Preprocessing or Data Preparation is a data mining technique that transforms raw data into an understandable format for ML algorithms. Real-world data is usually noisy (contains errors, outliers, duplicates), incomplete (some values are missed), and
 could be stored in different places and different formats. The task of Data Preprocessing is to handle these issues.  
The dataset contains both numerical and textual values, along with outliers and null values. These inconsistencies can disrupt clustering accuracy. Normalize the data extensively and provide detailed explanations for each normalization step in the report file.

### Load Data

In [245]:
diabetic_data = pd.read_csv('dataset_diabetes/diabetic_data.csv')

In [246]:
ids_mapping = pd.read_csv('dataset_diabetes/IDs_mapping.csv')
ids_mapping = ids_mapping.rename(columns={'admission_type_id': 'id'})
def split_dataframe_on_empty_row(df):
    empty_row_indices = df.index[df.isnull().all(axis=1)].tolist() 
    sections = []
    start_idx = 0
    for idx in empty_row_indices:
        sections.append(df.iloc[start_idx:idx].reset_index(drop=True))
        start_idx = idx + 2
    sections.append(df.iloc[start_idx:].reset_index(drop=True))
    return sections

sections = split_dataframe_on_empty_row(ids_mapping)

sections = [section for section in sections if not section.empty]

admission_type_mapping = sections[0].rename(columns={'id': 'admission_type_id', 'description': 'admission_type_desc'})
discharge_disposition_mapping = sections[1].rename(columns={'id': 'discharge_disposition_id', 'description': 'discharge_disposition_desc'})
admission_source_mapping = sections[2].rename(columns={'id': 'admission_source_id', 'description': 'admission_source_desc'})

# admission_type_mapping.to_csv('dataset_diabetes/admission_type_mapping.csv', index=False)
# discharge_disposition_mapping.to_csv('dataset_diabetes/discharge_disposition_mapping.csv', index=False)
# admission_source_mapping.to_csv('dataset_diabetes/admission_source_mapping.csv', index=False)

In [247]:
del ids_mapping

### Get basic information about each DataFrame

In [240]:
diabetic_data.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [181]:
diabetic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [182]:
admission_type_mapping

,admission_type_id,admission_type_desc
0,1,Emergency
1,2,Urgent
2,3,Elective
3,4,Newborn
4,5,Not Available
5,6,NaN
6,7,Trauma Center
7,8,Not Mapped


In [183]:
discharge_disposition_mapping

,discharge_disposition_id,discharge_disposition_desc
0,1,Discharged to home
1,2,Discharged/transferred to another short term h...
2,3,Discharged/transferred to SNF
3,4,Discharged/transferred to ICF
4,5,Discharged/transferred to another type of inpa...
5,6,Discharged/transferred to home with home healt...
6,7,Left AMA
7,8,Discharged/transferred to home under care of H...
8,9,Admitted as an inpatient to this hospital
9,10,Neonate discharged to another hospital for neo...


In [184]:
admission_source_mapping

,admission_source_id,admission_source_desc
0,1,Physician Referral
1,2,Clinic Referral
2,3,HMO Referral
3,4,Transfer from a hospital
4,5,Transfer from a Skilled Nursing Facility (SNF)
5,6,Transfer from another health care facility
6,7,Emergency Room
7,8,Court/Law Enforcement
8,9,Not Available
9,10,Transfer from critial access hospital


### Map the IDs to descriptions in diabetic_data

In [241]:
admission_type_mapping['admission_type_id'] = admission_type_mapping['admission_type_id'].astype('int64')
discharge_disposition_mapping['discharge_disposition_id'] = discharge_disposition_mapping['discharge_disposition_id'].astype('int64')
admission_source_mapping['admission_source_id'] = admission_source_mapping['admission_source_id'].astype('int64')

In [242]:
diabetic_data = diabetic_data.merge(admission_type_mapping, on='admission_type_id', how='left')
diabetic_data = diabetic_data.merge(discharge_disposition_mapping, on='discharge_disposition_id', how='left')
diabetic_data = diabetic_data.merge(admission_source_mapping, on='admission_source_id', how='left')

### Handling Missing and Invalid Values 

In [243]:
def find_invalid_values(df):
    placeholder_values = ["Unknown/Invalid", "NaN", "Not Available", "Not Mapped", "?"]
    
    placeholder_counts = {}
    
    for column in df.columns:
        placeholder_counts[column] = {}
        placeholder_counts[column]['NaN'] = df[column].isnull().sum()
        for value in placeholder_values:
            placeholder_counts[column][value] = df[column].astype(str).str.contains(re.escape(value), case=False, na=False).sum()
    
    placeholder_counts_df = pd.DataFrame(placeholder_counts).transpose()
    filtered_placeholder_counts_df = placeholder_counts_df[(placeholder_counts_df.T != 0).any()]
    percentage_df = (filtered_placeholder_counts_df / total_rows) * 100
    percentage_df["sum"] = percentage_df["Unknown/Invalid"] +  percentage_df["NaN"] + percentage_df["Not Available"] + percentage_df["Not Mapped"] +  percentage_df["?"]
    return percentage_df

In [244]:
find_invalid_values(diabetic_data)

,NaN,Unknown/Invalid,Not Available,Not Mapped,?,sum
race,0.000000,0.000000,0.000000,0.000000,2.233555,2.233555
gender,0.000000,0.002948,0.000000,0.000000,0.000000,0.002948
weight,0.000000,0.000000,0.000000,0.000000,96.858479,96.858479
payer_code,0.000000,0.000000,0.000000,0.000000,39.557416,39.557416
medical_specialty,0.000000,0.000000,0.000000,0.000000,49.082208,49.082208
diag_1,0.000000,0.000000,0.000000,0.000000,0.020636,0.020636
diag_2,0.000000,0.000000,0.000000,0.000000,0.351787,0.351787
diag_3,0.000000,0.000000,0.000000,0.000000,1.398306,1.398306
max_glu_serum,94.746772,0.000000,0.000000,0.000000,0.000000,94.746772
A1Cresult,83.277322,0.000000,0.000000,0.000000,0.000000,83.277322


In [227]:
# placeholders = ["Unknown/Invalid", "NaN", "Not Available", "Not Mapped", "?"]
# diabetic_data.replace(placeholders, np.nan, inplace=True)


##  2. Dimensionality Reduction

Some data mining algorithms, like K-Means, struggle with accurately clustering data when confronted with numerous features, leading to high dimensionality. This issue isn't exclusive to datasets with hundreds or thousands of features; even just ten features can pose accuracy challenges.  
Feature or dimensionality reduction aims to address this by transforming the original feature set into a smaller set of derived features that retain most of the original information.  
Principal Component Analysis (PCA) is a widely used technique for feature reduction. It condenses the original dataset into a set number of features known as principal components. The desired number of principal components must be specified.  
In this section, utilize PCA to reduce the dimensionality of the dataset. This approach is recommended if there are numerous variables, manual variable selection is impractical, or segmentation results are unsatisfactory.

##  3. Unsupervised Learning

Utilizing the **silhouette method**, determine the optimal `number of clusters` for the **K-means** method and the optimal input parameters `(minPnt, eps)` for the **DBSCAN** method.  
Based on the obtained values, store the best result from each method in a CSV file containing only the columns: `id_encounter`, `kmean_label`, and `dbscan_label`.

###  Silhouette Method
The Silhouette Method is a method to find the optimal number of clusters and interpretation and validation of consistency within clusters of data. The silhouette method computes silhouette coefficients of each point that measure how much a point is similar to its cluster compared to other clusters. by providing a succinct graphical representation of how well each object has been classified

### K-Means
K-Means Clustering is a type of Unsupervised Machine Learning algorithm that organizes an unlabeled dataset into distinct clusters. This method assigns data points to one of the K clusters based on their proximity to the cluster centers. Initially, cluster centroids are randomly placed in space. Then, each data point is assigned to the nearest cluster centroid. Subsequently, new cluster centroids are calculated. This iterative process continues until it converges on well-defined clusters.

### DBSCAN
DBSCAN is an unsupervised clustering algorithm, offering an alternative to KMeans and hierarchical clustering. It relies on two key parameters: Epsilon (ɛ), defining the neighborhood radius, and Minimum Points (minPts), specifying the minimum number of points to form a cluster. Epsilon determines the similarity threshold between points, influencing cluster size, while minPts affects cluster robustness and noise handling.  
Balancing these parameters is crucial for effective clustering without splitting valid clusters or aggregating unrelated points

# Questions

#### 1. What preprocessing steps did you perform on the dataset? Provide clear reasons for each decision made.

#### 2. What portion of the dataset did you retain during dimensionality reduction, and which variables were retained? Could you elaborate on the rationale behind this decision?

#### 3. Include a plot illustrating the silhouette coefficient plotted against the input parameters for each clustering method within the report file.

#### 4. How can we determine the optimal number of clusters in K-Means?

#### 5. How can we determine the optimal epsilon value and minPts in DBSCAN?

#### 6. When would you recommend using K-Means, and when would you suggest using DBSCAN instead